In [12]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tag import StanfordNERTagger
%matplotlib inline

In [13]:
fields = ['Sentence','Word', 'POS', 'Tag']

train = pd.read_csv("./data/TrainNER.csv",sep=';',na_values='', encoding='latin1')

test1 = pd.read_csv("./data/Test1NER.csv",sep=';',na_values='', encoding='latin1')
test2 = pd.read_csv("./data/Test2NER.csv",sep=';',na_values='', encoding='latin1')

#Add columns to test1 df
test1.loc[-1] = [test1.columns[0], test1.columns[1], test1.columns[2]]  # rename columns
test1.index = test1.index + 1  # shifting index
test1 = test1.sort_index()  # sorting by index
test1 = test1.rename(columns={test1.columns[0]: 'Sentences', test1.columns[1]: 'Word',test1.columns[2]:  'POS'}) # adding first row

#Add columns to test2 df
test2.loc[-1] = [test2.columns[0], test2.columns[1]]  # rename columns
test2.index = test2.index + 1  # shifting index
test2 = test2.sort_index()  # sorting by index
test2 = test2.rename(columns={test2.columns[0]: 'Sentences', test2.columns[1]: 'Word'}) # adding first row

test1_format = pd.read_csv("./data/Test1NER_PredictedFormat.csv",sep=';',na_values='', encoding='latin1')
test2_format = pd.read_csv("./data/Test2NER_PredictedFormat.csv",sep=';',na_values='', encoding='latin1')

#print(test1.iloc[0:2])

In [5]:
# Stanford NER tagger    
def stanford_tagger(token_text):
	st = StanfordNERTagger('./stanford-ner-4.0.0/classifiers/english.muc.7class.distsim.crf.ser.gz',
							'./stanford-ner-4.0.0/stanford-ner-4.0.0.jar',
							encoding='utf-8')   
	ne_tagged = st.tag(token_text)
	return(ne_tagged)

In [6]:
def bio_tagger(ne_tagged):
		bio_tagged = []
		prev_tag = "O"
		for token, tag in ne_tagged:
			if tag == "O": #O
				bio_tagged.append((token, tag))
				prev_tag = tag
				continue
			if tag != "O" and prev_tag == "O": # Begin NE
				bio_tagged.append((token, "B-"+tag))
				prev_tag = tag
			elif prev_tag != "O" and prev_tag == tag: # Inside NE
				bio_tagged.append((token, "I-"+tag))
				prev_tag = tag
			elif prev_tag != "O" and prev_tag != tag: # Adjacent NE
				bio_tagged.append((token, "B-"+tag))
				prev_tag = tag
		return bio_tagged

In [7]:
#test1
only_words = list()
for w in test1['Word']:
    if pd.notna(w):
        only_words.append(w)
#print(only_words)

tokens,ne_tags = zip(*bio_tagger(stanford_tagger(only_words)))

test1["Predicted"] = ne_tags

#test2
only_words = list()
for w in test2['Word']:
    if pd.notna(w):
        only_words.append(w)
#print(only_words)

tokens,ne_tags = zip(*bio_tagger(stanford_tagger(only_words)))

test2["Predicted"] = ne_tags
#print(structure_ne(stanford_tree(bio_tagger(stanford_tagger(only_words)))))

In [10]:
#for submission
for w in test1.index:
    if test1.loc[w,'Predicted']=='B-LOCATION':
        test1.loc[w,'Predicted']='B-geo'
    if test1.loc[w,'Predicted']=='I-LOCATION':
        test1.loc[w,'Predicted']='I-geo'

    if test1.loc[w,'Predicted']=='B-PERSON':
        test1.loc[w,'Predicted']='B-per'
    if test1.loc[w,'Predicted']=='I-PERSON':
        test1.loc[w,'Predicted']='I-per'

    if test1.loc[w,'Predicted']=='B-DATE':
        test1.loc[w,'Predicted']='O'
    if test1.loc[w,'Predicted']=='I-DATE':
        test1.loc[w,'Predicted']='O'

    if test1.loc[w,'Predicted']=='B-ORGANIZATION':
        test1.loc[w,'Predicted']='B-org'
    if test1.loc[w,'Predicted']=='I-ORGANIZATION':
        test1.loc[w,'Predicted']='I-org'

    if test1.loc[w,'Predicted']=='B-TIME':
        test1.loc[w,'Predicted']='B-time'
    if test1.loc[w,'Predicted']=='I-TIME':
        test1.loc[w,'Predicted']='I-time'

    if test1.loc[w,'Predicted']=='B-MONEY':
        test1.loc[w,'Predicted']='O'
    if test1.loc[w,'Predicted']=='I-MONEY':
        test1.loc[w,'Predicted']='O'

    if test1.loc[w,'Predicted']=='B-PERCENT':
        test1.loc[w,'Predicted']='O'
    if test1.loc[w,'Predicted']=='I-PERCENT':
        test1.loc[w,'Predicted']='O'


test1.to_csv("Submission1_Stanford.csv",index=False,sep = ';')

In [11]:
#submission
for w in test2.index:
    if test2.loc[w,'Predicted']=='B-LOCATION':
        test2.loc[w,'Predicted']='B-geo'
    if test2.loc[w,'Predicted']=='I-LOCATION':
        test2.loc[w,'Predicted']='I-geo'

    if test2.loc[w,'Predicted']=='B-PERSON':
        test2.loc[w,'Predicted']='B-per'
    if test2.loc[w,'Predicted']=='I-PERSON':
        test2.loc[w,'Predicted']='I-per'

    if test2.loc[w,'Predicted']=='B-DATE':
        test2.loc[w,'Predicted']='O'
    if test2.loc[w,'Predicted']=='I-DATE':
        test2.loc[w,'Predicted']='O'

    if test2.loc[w,'Predicted']=='B-ORGANIZATION':
        test2.loc[w,'Predicted']='B-org'
    if test2.loc[w,'Predicted']=='I-ORGANIZATION':
        test2.loc[w,'Predicted']='I-org'

    if test2.loc[w,'Predicted']=='B-TIME':
        test2.loc[w,'Predicted']='B-time'
    if test2.loc[w,'Predicted']=='I-TIME':
        test2.loc[w,'Predicted']='I-time'

    if test2.loc[w,'Predicted']=='B-MONEY':
        test2.loc[w,'Predicted']='O'
    if test2.loc[w,'Predicted']=='I-MONEY':
        test2.loc[w,'Predicted']='O'

    if test2.loc[w,'Predicted']=='B-PERCENT':
        test2.loc[w,'Predicted']='O'
    if test2.loc[w,'Predicted']=='I-PERCENT':
        test2.loc[w,'Predicted']='O'


test2.to_csv("Submission2_Stanford.csv",index=False,sep = ';')